<a href="https://colab.research.google.com/github/rnbinama/CFG2024/blob/main/CFGPresentation(NewAPIKeys).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import random
import csv

# Function to search recipes based on ingredient and dietary requirement
def recipe_search(ingredient, diet=None):
    app_id = '862fe5be'  # Edamam account app_id
    app_key = 'a97390f104a0d2559358c27ca87014b1'
    user_id = 'natbatbin'

    # Base URL
    url = 'https://api.edamam.com/api/recipes/v2'

    # Request parameters
    params = {
        'type': 'public',
        'q': ingredient,
        'app_id': app_id,
        'app_key': app_key
    }

    if diet:
        for d in diet:
            params.setdefault('health', []).append(d)

    headers = {
        'Edamam-Account-User': user_id  # REQUIRED header
    }

    result = requests.get(url, params=params, headers=headers)

    # Check for errors before trying to parse
    if result.status_code != 200:
        print(f"Error fetching data for {ingredient}: {result.status_code}")
        print(result.text)
        return []

    data = result.json()

    if 'hits' not in data:
        print(f"No recipes found for {ingredient}")
        return []

    recipes = []
    for hit in data['hits']:
        recipe = hit['recipe']
        recipes.append({
            'label': recipe['label'],
            'uri': recipe['url'],  # Actual URL for instructions
            'ingredients': recipe['ingredientLines'],
            'instructions': ['Please follow the link for full instructions.']
        })

    return recipes

# def recipe_category(ingredient, diet=None):
    meal_types = 'lunch/dinner'
    results = recipe_search(ingredient, diet, meal_types)

    return results

# Function to run the recipe planner

def run():
    dietary_options = {
        '1': 'lactose-free',
        '2': 'gluten-free',
        '3': 'vegetarian',
        '4': 'vegan',
        '5': 'kosher',
        '6': 'keto-friendly',
        '7': 'sugar-conscious',
        '8': 'dairy-free'
    }

# Get dietary requirement
# Prompt asks user about dietary requirements
# Then it loops through the dietary_options dictionary we created and prints each option
# with its corresponding number the option replace function gives it dashes, capitalise
# just the layout format, numbered list

# Then if choices are entered users input are split by commas
# then num.strip removes any whitespace and checks whether the number input is a
# Valid key in 'dietary_options'
# If it is, it gets the corresponding requirement from the dietary dictionary
# It then creates a list of all the requirements selected and inputs them into url

    print("Do you have any dietary requirements? Please enter the numbers separated by commas (or press Enter to skip):")
    for num, option in dietary_options.items():
        print(f"{num}. {option.replace('-', ' ').capitalize()}")

    dietary_choice = input("Enter the number/numbers corresponding to your dietary requirements (e.g., 1,3,5): ")
    if dietary_choice:
        diet = [dietary_options.get(num.strip()) for num in dietary_choice.split(',') if num.strip() in dietary_options]
    else:
        diet = None

    # Get ingredients
    print("\nPlease enter your ingredient choices when prompted (e.g., 'potato', 'cheese', 'chicken' & 'tomato')")

    ingredients = []
    for i in range(1, 4):
        ingredient = input(f"Ingredient{i}: ")
        ingredients.append(ingredient)

    all_recipes = {}

    # Display recipes for each ingredient
    for ingredient in ingredients:
        print(f"\n{ingredient.upper()} RECIPES:\n" + "-" * 30)
        recipes = recipe_search(ingredient, diet)
        all_recipes[ingredient] = recipes

        if recipes:
            all_recipes[ingredient] = recipes
            for recipe in recipes:
                print(f"Recipe: {recipe['label']}")
                print(f"Weblink: {recipe['uri']}")
                print("Ingredients:")
                for ing in recipe['ingredients']:
                    print(f"{ing}")
                print()
        else:
            print(f"Sorry, no recipes available for {ingredient} with the given dietary requirements.")
            all_recipes[ingredient] = []

    return all_recipes


# Function to generate random meal plan for the week
def generate_meal_plan(recipes):
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    meal_plan = {}

    for day in days_of_week:
        random_ingredient = random.choice(list(recipes.keys()))
        if recipes[random_ingredient]:
            random_recipe = random.choice(recipes[random_ingredient])
            meal_plan[day] = random_recipe
        else:
            meal_plan[day] = {"label": "No suitable recipe found", "uri": "", "ingredients": []}
    return meal_plan


# Function to create grocery list from meal plan
def create_grocery_list(meal_plan):
    grocery_list = []

    for day, recipe in meal_plan.items():
        grocery_list.extend(recipe['ingredients'])

    return list(set(grocery_list))  # Remove duplicates


# Function to write meal plan and grocery list to a file
import csv

def write_to_file(meal_plan, grocery_list):
    with open('meal_plan_and_grocery_list.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)

        csvwriter.writerow(['Day', 'Meal', 'Ingredients', 'Instructions', 'Link'])

        for day, recipe in meal_plan.items():
            ingredients = ", ".join(recipe['ingredients'])
            instructions = " | ".join(f"{i+1}. {instruction}" for i, instruction in enumerate(recipe.get('instructions', ['No instructions available'])))
            label = recipe['label']
            url = recipe['uri']

            csvwriter.writerow([day, label, ingredients, instructions, url])

        csvwriter.writerow([" ", " ", " ", " ", " "])

        csvwriter.writerow(['Grocery List'])
        for item in grocery_list:
            csvwriter.writerow([" ",item])

    print("\nMeal plan and grocery list have been saved to 'meal_plan_and_grocery_list.csv'.")


# Main flow
def main():
    print("""Hello and welcome to our recipe planner!
             You will first be asked whether you have any dietary requirements.
             You will then be prompted three times to enter a singular ingredient to create a wonderful list of delicious recipes.
             A randomized meal plan for the week will then be generated and a grocery list to go alongside.
             This information will be displayed conveniently in a csv file for you!

             Have Fun and Happy Cooking!\n""")
    input("Press 'Enter' to continue...\n")
    recipes = run()

    if any(recipes.values()):  # Check if any recipes were found
        print("\nGenerating random meal plan for the week...")
        input("Press 'Enter' to continue...")
        meal_plan = generate_meal_plan(recipes)

        print("\nYour Generated Meal Plan for the Week:")
        for day, recipe in meal_plan.items():
            print(f"{day}: {recipe['label']}")


        print("\nCreating grocery list...")
        input("Press Enter to continue...")
        grocery_list = create_grocery_list(meal_plan)

        write_to_file(meal_plan, grocery_list)
        print("\n Enjoy your meals!")
    else:
        print("\nNo recipes found matching your dietary requirements and ingredients. Please try again with different inputs.")

# Run the program
main()

Hello and welcome to our recipe planner!
             You will first be asked whether you have any dietary requirements.
             You will then be prompted three times to enter a singular ingredient to create a wonderful list of delicious recipes.
             A randomized meal plan for the week will then be generated and a grocery list to go alongside.
             This information will be displayed conveniently in a csv file for you!

             Have Fun and Happy Cooking!

Press 'Enter' to continue...

Do you have any dietary requirements? Please enter the numbers separated by commas (or press Enter to skip):
1. Lactose free
2. Gluten free
3. Vegetarian
4. Vegan
5. Kosher
6. Keto friendly
7. Sugar conscious
8. Dairy free
Enter the number/numbers corresponding to your dietary requirements (e.g., 1,3,5): 

Please enter your ingredient choices when prompted (e.g., 'potato', 'cheese', 'chicken' & 'tomato')
Ingredient1: rice
Ingredient2: salmon
Ingredient3: cheese

RICE RECIPES:
--